In [1]:
import pandas as pd
from scipy import sparse

In [3]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
print(ratings.shape)
ratings.head()

(100836, 4)


,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [4]:
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)

Before:  (610, 9719)
After:  (610, 2269)


In [5]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(100)

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,0.063117,-0.023768,0.143482,0.011998,0.087931,0.224052,0.034223,0.009277,0.008331,...,0.017477,0.032470,0.134701,0.153158,0.101301,0.049897,0.003233,0.187953,0.062174,0.353194
(500) Days of Summer (2009),0.063117,1.000000,0.142471,0.273989,0.193960,0.148903,0.142141,0.159756,0.135486,0.200135,...,0.374515,0.178655,0.068407,0.414585,0.355723,0.252226,0.216007,0.053614,0.241092,0.125905
10 Cloverfield Lane (2016),-0.023768,0.142471,1.000000,-0.005799,0.112396,0.006139,-0.016835,0.031704,-0.024275,0.272943,...,0.242663,0.099059,-0.023477,0.272347,0.241751,0.195054,0.319371,0.177846,0.096638,0.002733
10 Things I Hate About You (1999),0.143482,0.273989,-0.005799,1.000000,0.244670,0.223481,0.211473,0.011784,0.091964,0.043383,...,0.243118,0.104858,0.132460,0.091853,0.158637,0.281934,0.050031,0.121029,0.130813,0.110612
"10,000 BC (2008)",0.011998,0.193960,0.112396,0.244670,1.000000,0.234459,0.119132,0.059187,-0.025882,0.089328,...,0.260261,0.087592,0.094913,0.184521,0.242299,0.240231,0.094773,0.088045,0.203002,0.083518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Almost Famous (2000),0.099554,0.209549,0.032088,0.296727,0.134434,0.118628,0.242958,0.079158,0.005092,0.051279,...,0.244619,0.085395,0.072505,0.221259,0.126008,0.362571,0.011577,0.208008,0.186599,0.147413
Along Came Polly (2004),0.027287,0.282426,0.113213,0.193085,0.162678,0.180259,0.112928,0.121704,0.125792,0.124032,...,0.173133,0.160430,0.029076,0.189165,0.166278,0.309183,0.078468,-0.036498,0.231566,0.025928
Along Came a Spider (2001),0.064762,-0.003205,0.016372,0.085365,-0.018241,0.080388,0.094016,-0.016678,0.079375,-0.028052,...,0.115347,0.093774,0.085286,0.150757,0.052144,0.174489,0.014189,0.025327,0.233244,0.043581


In [6]:
def get_similar(movie_name,rating):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [11]:
Thriller_hunt = [("Devil in a Blue Dress (1995)",5),("Hackers (1995)",3),("Aliens (1986)",1)]
similar_movies = pd.DataFrame()
for movie,rating in Thriller_hunt:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)

C:\Users\bhara\AppData\Local\Temp\ipykernel_17280\901554589.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\bhara\AppData\Local\Temp\ipykernel_17280\901554589.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\bhara\AppData\Local\Temp\ipykernel_17280\901554589.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)


title,Devil in a Blue Dress (1995),"Killer, The (Die xue shuang xiong) (1989)","Thin Blue Line, The (1988)",Midnight Run (1988),"Siege, The (1998)",Bob Roberts (1992),Cop Land (1997),Cape Fear (1991),Cape Fear (1962),Breakdown (1997),...,Life Is Beautiful (La Vita è bella) (1997),Shrek 2 (2004),Harry Potter and the Prisoner of Azkaban (2004),Shrek (2001),Eternal Sunshine of the Spotless Mind (2004),"Lord of the Rings: The Two Towers, The (2002)",Fight Club (1999),Inception (2010),"Dark Knight, The (2008)","Lord of the Rings: The Fellowship of the Ring, The (2001)"
0,2.50000,0.965269,0.915103,0.822774,0.820350,0.815009,0.803355,0.770324,0.760916,0.743771,...,-0.148147,-0.151510,-0.155323,-0.155593,-0.156714,-0.159447,-0.161780,-0.165866,-0.170552,-0.173112
1,0.05734,0.046208,0.001659,0.070462,0.070798,0.058523,0.106360,0.102891,0.042374,0.051871,...,0.029633,0.083829,0.033298,0.043800,0.041935,0.026585,0.026675,0.005455,0.006651,0.032809
2,-0.28578,-0.413181,-0.209885,-0.337705,-0.287661,-0.262364,-0.413649,-0.415075,-0.425392,-0.234857,...,-0.074328,-0.244680,-0.122613,-0.270327,-0.151300,-0.313814,-0.355261,-0.120046,-0.174020,-0.303829


In [13]:
similar_movies.sum().sort_values(ascending=False).head(20)

title
Devil in a Blue Dress (1995)                 2.271560
Thin Blue Line, The (1988)                   0.706877
Dolores Claiborne (1995)                     0.617949
Bob Roberts (1992)                           0.611168
Something to Talk About (1995)               0.609428
Siege, The (1998)                            0.603487
Killer, The (Die xue shuang xiong) (1989)    0.598296
Blown Away (1994)                            0.594321
Once Were Warriors (1994)                    0.588517
Nobody's Fool (1994)                         0.574649
City Hall (1996)                             0.571821
Victor/Victoria (1982)                       0.571056
Corrina, Corrina (1994)                      0.569125
Mighty Aphrodite (1995)                      0.561369
Breakdown (1997)                             0.560785
Midnight Run (1988)                          0.555531
Hackers (1995)                               0.548714
Disclosure (1994)                            0.540392
Paper, The (1994)     

In [9]:
action_lover = [("Amazing Spider-Man, The (2012)",5),("Mission: Impossible III (2006)",4),("Toy Story 3 (2010)",2),("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4)]
similar_movies = pd.DataFrame()
for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(20)

C:\Users\bhara\AppData\Local\Temp\ipykernel_17280\1191409001.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\bhara\AppData\Local\Temp\ipykernel_17280\1191409001.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\bhara\AppData\Local\Temp\ipykernel_17280\1191409001.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\bhara\AppData\Local\Temp\ipykernel_17280\1191409001.py:4: FutureWarning: The frame.append method is deprecated and will be remo

title
Amazing Spider-Man, The (2012)                           3.233134
Mission: Impossible III (2006)                           2.874798
2 Fast 2 Furious (Fast and the Furious 2, The) (2003)    2.701477
Over the Hedge (2006)                                    2.229721
Crank (2006)                                             2.176259
Mission: Impossible - Ghost Protocol (2011)              2.159666
Hancock (2008)                                           2.156098
The Amazing Spider-Man 2 (2014)                          2.153677
Hellboy (2004)                                           2.137518
Snakes on a Plane (2006)                                 2.137396
Jumper (2008)                                            2.129716
Chronicles of Riddick, The (2004)                        2.121689
Tron: Legacy (2010)                                      2.111843
Fantastic Four (2005)                                    2.083022
X-Men: The Last Stand (2006)                             2.077530
Wrec